<a href="https://colab.research.google.com/github/sutharimanikanta/-technity-tasks-/blob/main/myliver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 1.8 MB/s eta 0:00:00


In [ ]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")


model imports

In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.decomposition import FactorAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from skopt import gp_minimize
from skopt.space import Integer, Real
import itertools
from sklearn.utils import resample
# Load dataset
data = pd.read_csv('/content/Indian Liver Patient Dataset (ILPD).csv')
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['gender'] = label_encoder.fit_transform(data['gender'])
X = data.drop(columns=['is_patient'])
y = data['is_patient']
# Count the number of positive and negative cases
num_positive_cases = (y == 1).sum()
num_negative_cases = (y == 0).sum()

In [ ]:
# Calculate the desired number of samples for each class
desired_num_samples = max(num_positive_cases, num_negative_cases)

import itertools

# Define all possible methods and their combinations
methods = ['L1', 'L2', 'Bayesian']  # Adjust as needed
all_combinations = []

# Generate all subsets and permutations of methods
for r in range(1, len(methods) + 1):
    for subset in itertools.combinations(methods, r):
        all_combinations.append(subset)

# Add reverse combinations
for combination in all_combinations.copy():
    if len(combination) > 1:
        for r in range(1, len(combination)):
            for reverse_subset in itertools.combinations(combination, r):
                if reverse_subset not in all_combinations:
                    all_combinations.append(reverse_subset)


In [ ]:

def preprocess_data(X, y):
    # Resample the data to balance the classes
    ros = RandomOverSampler(sampling_strategy='auto', random_state=42)
    X_resampled, y_resampled = ros.fit_resample(X, y)

    # Change class label 2 to 0 in y_resampled (assuming binary classification)
    y_resampled[y_resampled == 2] = 0

    # Creating a new DataFrame with the resampled data
    resampled_data = pd.DataFrame(X_resampled, columns=X.columns)
    resampled_data['is_patient'] = y_resampled

    # Impute missing values with the median
    imputer = SimpleImputer(strategy='median')
    data_imputed = pd.DataFrame(imputer.fit_transform(resampled_data), columns=resampled_data.columns)

    # Detect and handle outliers
    z_scores = np.abs((data_imputed - data_imputed.mean()) / data_imputed.std())
    outliers = (z_scores > 3)
    for column in data_imputed.columns:
        median = data_imputed[column].median()
        data_imputed.loc[outliers[column], column] = median

    # Remove the 'is_patient' column before scaling
    y_resampled = data_imputed['is_patient']
    data_no_outliers = data_imputed.drop(columns=['is_patient'])

    # Standardize the data
    scaler = StandardScaler()
    data_scaled = pd.DataFrame(scaler.fit_transform(data_no_outliers), columns=data_no_outliers.columns)
    fa = FactorAnalysis(n_components=10)  # Adjust as needed
    X_fa = fa.fit_transform(data_scaled)

# Apply Linear Discriminant Analysis (LDA)
    lda = LDA(n_components=1)
    X_lda = lda.fit_transform(data_scaled, y_resampled)
    # Convert the FA and LDA results to DataFrames
    df_fa = pd.DataFrame(X_fa, columns=[f'FA_Component_{i+1}' for i in range(X_fa.shape[1])])
    df_lda = pd.DataFrame(X_lda, columns=['LDA_Component'])

    # Integrate the datasets by concatenating them along the columns
    final_data = pd.concat([df_fa, df_lda], axis=1)

    return final_data, y_resampled
final_data, y_resampled=preprocess_data(X, y)

In [ ]:
# !pip install pymc3

In [ ]:
import numpy as np
import pandas as pd
# import pymc3 as pm
import matplotlib.pyplot as plt
import arviz as az
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from skopt import gp_minimize
from skopt.space import Integer, Real
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FactorAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from imblearn.over_sampling import RandomOverSampler
import itertools
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
from sklearn.linear_model import Ridge, Lasso
import numpy as np
from skopt import gp_minimize
from skopt.space import Real, Integer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd
# FGSM function for adversarial examples
def fgsm_attack(model, criterion, data, target, epsilon):
    data.requires_grad = True
    output = model(data)
    loss = criterion(output, target)
    model.zero_grad()
    loss.backward()
    data_grad = data.grad.data
    sign_data_grad = data_grad.sign()
    perturbed_data = data + epsilon * sign_data_grad
    return perturbed_data

def perform_l2_regularization(final_data, y_resampled):
    ridge = Ridge(alpha=0.1)  # You can adjust the alpha value as needed
    ridge.fit(final_data, y_resampled)
    selected_features_indices = np.where(ridge.coef_ != 0)[0]
    selected_features_names = final_data.columns[selected_features_indices].tolist()
    return selected_features_names

def perform_l1_regularization(final_data, y_resampled):
    lasso = Lasso(alpha=0.1)
    lasso.fit(final_data, y_resampled)
    selected_features_indices = np.where(lasso.coef_ != 0)[0]
    selected_features_names = final_data.columns[selected_features_indices].tolist()
    return selected_features_names

def perform_bayesian_optimization(final_data, y_resampled):
    n_features = final_data.shape[1]
    search_space = [Integer(0, 1) for _ in range(n_features)] + [Real(1e-6, 1e+6, prior='log-uniform')]

    def objective(params):
        selected_features, C = params[:-1], params[-1]
        selected_features = np.array(selected_features, dtype=bool)
        X_selected = final_data.iloc[:, selected_features]
        if X_selected.shape[1] == 0:
            X_selected = final_data
        model = LogisticRegression(C=C, penalty='l2', solver='liblinear', random_state=42)
        scores = cross_val_score(model, X_selected, y_resampled, cv=5, scoring='accuracy')
        return -np.mean(scores)

    result = gp_minimize(
        func=objective,
        dimensions=search_space,
        n_calls=50,
        random_state=42
    )

    best_features = np.array(result.x[:-1], dtype=bool)
    best_C = result.x[-1]
    selected_features_indices = np.arange(n_features)[best_features]
    selected_features_names = final_data.columns[selected_features_indices].tolist()

    return selected_features_names

def select_features(selected_methods, final_data, y_resampled):
    selected_features_names = []

    if 'L1' in selected_methods:
        l1_selected_features = perform_l1_regularization(final_data, y_resampled)
        selected_features_names.extend(l1_selected_features)
        print(f"Features selected by L1: {l1_selected_features}")

    if 'L2' in selected_methods:
        l2_selected_features = perform_l2_regularization(final_data, y_resampled)
        selected_features_names.extend(l2_selected_features)
        print(f"Features selected by L2: {l2_selected_features}")

    if 'Bayesian' in selected_methods:
        bayesian_selected_features = perform_bayesian_optimization(final_data, y_resampled)
        selected_features_names.extend(bayesian_selected_features)
        print(f"Features selected by Bayesian: {bayesian_selected_features}")

    return selected_features_names

# Function to evaluate model performance
def evaluate_model(model, X_train, X_test, y_train, y_test, epsilon=0.1):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

    model.train()
    for epoch in range(5):  # Number of epochs can be adjusted
        optimizer.zero_grad()
        output = model(X_train_tensor)
        loss = criterion(output, y_train_tensor)
        loss.backward()
        optimizer.step()

        adv_data = fgsm_attack(model, criterion, X_train_tensor, y_train_tensor, epsilon)

        optimizer.zero_grad()
        adv_output = model(adv_data)
        adv_loss = criterion(adv_output, y_train_tensor)
        adv_loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        output = model(X_test_tensor)
        y_pred = output.argmax(dim=1).numpy()
        y_proba = nn.Softmax(dim=1)(output).numpy()[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    return accuracy, precision, recall, f1, auc



In [ ]:
!pip install lazypredict

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyRegressor

In [ ]:
# Evaluate neural network model performance with adversarial training
def evaluate_nn_model(model, X_train, X_test, y_train, y_test, epsilon=0.1):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

    model.train()
    for epoch in range(5):  # Number of epochs can be adjusted
        optimizer.zero_grad()
        output = model(X_train_tensor)
        loss = criterion(output, y_train_tensor)
        loss.backward()
        optimizer.step()

        adv_data = fgsm_attack(model, criterion, X_train_tensor, y_train_tensor, epsilon)

        optimizer.zero_grad()
        adv_output = model(adv_data)
        adv_loss = criterion(adv_output, y_train_tensor)
        adv_loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        output = model(X_test_tensor)
        y_pred = output.argmax(dim=1).numpy()
        y_proba = nn.Softmax(dim=1)(output).numpy()[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    return accuracy, precision, recall, f1, auc

# Evaluate non-neural network models
def evaluate_sklearn_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else np.zeros_like(y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    return accuracy, precision, recall, f1, auc

# Run pipeline with adversarial training for neural networks and standard evaluation for others
def run_pipeline(selected_methods, final_data, y_resampled, epsilon=0.1):
    selected_features_names = select_features(selected_methods, final_data, y_resampled)
    X_selected = final_data[selected_features_names] if selected_features_names else final_data.copy()

    X_train, X_test, y_train, y_test = train_test_split(X_selected, y_resampled, test_size=0.2, random_state=42)

    classifiers = {
        "Logistic Regression": LogisticRegression(solver='liblinear', random_state=42),
        "K-Nearest Neighbors": KNeighborsClassifier(),
        "Random Forest": RandomForestClassifier(random_state=42),
        "SVM": SVC(probability=True, random_state=42),
        "MLP": MLPClassifier(random_state=42)
    }

    param_grids = {
        "Logistic Regression": {
            'C': [0.001, 0.01, 0.1, 1, 10, 100],
            'penalty': ['l1', 'l2']
        },
        "K-Nearest Neighbors": {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance']
        },
        "Random Forest": {
            'n_estimators': [100, 200, 300],
            'max_depth': [None, 5, 10, 15]
        },
        "SVM": {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf']
        },
        "MLP": {
            'hidden_layer_sizes': [(50,), (100,), (50, 50)],
            'alpha': [0.0001, 0.001, 0.01]
        }
    }

    for name, clf in classifiers.items():
        param_grid = param_grids.get(name)
        if param_grid:
            grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
            grid_search.fit(X_train, y_train)
            best_params = grid_search.best_params_
            classifiers[name] = clf.set_params(**best_params)
            if name == "MLP":
                # Convert MLPClassifier to a PyTorch model for adversarial training
                class PyTorchMLP(nn.Module):
                    def __init__(self, input_size, hidden_layer_sizes, output_size):
                        super(PyTorchMLP, self).__init__()
                        self.hidden_layers = nn.ModuleList()
                        self.hidden_layers.append(nn.Linear(input_size, hidden_layer_sizes[0]))
                        for i in range(1, len(hidden_layer_sizes)):
                            self.hidden_layers.append(nn.Linear(hidden_layer_sizes[i-1], hidden_layer_sizes[i]))
                        self.output_layer = nn.Linear(hidden_layer_sizes[-1], output_size)
                        self.activation = nn.ReLU()

                    def forward(self, x):
                        for layer in self.hidden_layers:
                            x = self.activation(layer(x))
                        x = self.output_layer(x)
                        return x

                input_size = X_train.shape[1]
                hidden_layer_sizes = best_params['hidden_layer_sizes']
                output_size = len(np.unique(y_train))
                pytorch_mlp = PyTorchMLP(input_size, hidden_layer_sizes, output_size)
                results = evaluate_nn_model(pytorch_mlp, X_train, X_test, y_train, y_test, epsilon)
            else:
                results = evaluate_sklearn_model(clf, X_train, X_test, y_train, y_test)
            print(f"{name} after {selected_methods}: Accuracy={results[0]:.2f}, Precision={results[1]:.2f}, Recall={results[2]:.2f}, F1-Score={results[3]:.2f}, AUC={results[4]:.2f}")

    # Use LazyRegressor for regression tasks
    lazy_regressor = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None, random_state=42, regressors='all')
    models, predictions = lazy_regressor.fit(X_train, X_test, y_train, y_test)
    print(models)

    return classifiers

# Preprocess the data
# [Include your existing preprocess_data function here]

# All combinations of feature selection methods
methods = ['L1', 'L2', 'Bayesian']
all_combinations = []
for r in range(1, len(methods) + 1):
    all_combinations.extend(itertools.combinations(methods, r))

# Perform pipeline for each combination of methods
results = []
for combination in all_combinations:
    print(f"\nRunning pipeline for methods: {combination}")
    classifiers = run_pipeline(combination, final_data, y_resampled)
    results.append((combination, classifiers))

# Further process or display results as needed
for combination, classifiers in results:
    print("\nFinal Model Performances for combination:", combination)
    for name, clf in classifiers.items():
        print(f"{name}: {clf}")


Running pipeline for methods: ('L1',)
Features selected by L1: ['LDA_Component']
Logistic Regression after ('L1',): Accuracy=0.76, Precision=0.83, Recall=0.59, F1-Score=0.69, AUC=0.79
K-Nearest Neighbors after ('L1',): Accuracy=0.83, Precision=0.90, Recall=0.69, F1-Score=0.78, AUC=0.94
Random Forest after ('L1',): Accuracy=0.83, Precision=0.84, Recall=0.76, F1-Score=0.80, AUC=0.88
SVM after ('L1',): Accuracy=0.74, Precision=0.86, Recall=0.51, F1-Score=0.64, AUC=0.79
MLP after ('L1',): Accuracy=0.74, Precision=0.88, Recall=0.49, F1-Score=0.63, AUC=0.78


100%|██████████| 42/42 [00:02<00:00, 15.93it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000116 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 221
[LightGBM] [Info] Number of data points in the train set: 665, number of used features: 1
[LightGBM] [Info] Start training from score 0.512782
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

Logistic Regression after ('L2',): Accuracy=0.75, Precision=0.80, Recall=0.59, F1-Score=0.68, AUC=0.79
K-Nearest Neighbors after ('L2',): Accuracy=0.80, Precision=0.89, Recall=0.64, F1-Score=0.74, AUC=0.94
Random Forest after ('L2',): Accuracy=0.85, Precision=0.93, Recall=0.72, F1-Score=0.81, AUC=0.96
SVM after ('L2',): Accuracy=0.71, Precision=0.81, Recall=0.45, F1-Score=0.58, AUC=0.78
MLP after ('L2',): Accuracy=0.38, Precision=0.40, Recall=0.72, F1-Score=0.51, AUC=0.38


100%|██████████| 42/42 [00:02<00:00, 16.70it/s]


...

100%|██████████| 42/42 [00:00<00:00, 2562.67it/s]

Empty DataFrame
Columns: [Adjusted R-Squared, R-Squared, RMSE, Time Taken]
Index: []

Running pipeline for methods: ('L1', 'L2', 'Bayesian')
Features selected by L1: ['LDA_Component']
Features selected by L2: ['FA_Component_1', 'FA_Component_2', 'FA_Component_3', 'FA_Component_4', 'FA_Component_5', 'LDA_Component']


Features selected by Bayesian: ['FA_Component_2', 'FA_Component_3', 'FA_Component_4', 'FA_Component_5', 'FA_Component_6', 'FA_Component_7', 'LDA_Component']
Logistic Regression after ('L1', 'L2', 'Bayesian'): Accuracy=0.75, Precision=0.80, Recall=0.59, F1-Score=0.68, AUC=0.79
K-Nearest Neighbors after ('L1', 'L2', 'Bayesian'): Accuracy=0.80, Precision=0.89, Recall=0.64, F1-Score=0.74, AUC=0.94
Random Forest after ('L1', 'L2', 'Bayesian'): Accuracy=0.89, Precision=0.91, Recall=0.83, F1-Score=0.87, AUC=0.96
SVM after ('L1', 'L2', 'Bayesian'): Accuracy=0.74, Precision=0.85, Recall=0.52, F1-Score=0.64, AUC=0.79
MLP after ('L1', 'L2', 'Bayesian'): Accuracy=0.45, Precision=0.43, Recall=0.75, F1-Score=0.55, AUC=0.39


100%|██████████| 42/42 [00:00<00:00, 2789.38it/s]

Empty DataFrame
Columns: [Adjusted R-Squared, R-Squared, RMSE, Time Taken]
Index: []

Final Model Performances for combination: ('L1',)
Logistic Regression: LogisticRegression(C=0.001, random_state=42, solver='liblinear')
K-Nearest Neighbors: KNeighborsClassifier(weights='distance')
Random Forest: RandomForestClassifier(max_depth=15, n_estimators=200, random_state=42)
SVM: SVC(C=0.1, kernel='linear', probability=True, random_state=42)
MLP: MLPClassifier(hidden_layer_sizes=(50,), random_state=42)

Final Model Performances for combination: ('L2',)
Logistic Regression: LogisticRegression(C=0.1, penalty='l1', random_state=42, solver='liblinear')
K-Nearest Neighbors: KNeighborsClassifier(weights='distance')
Random Forest: RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42)
SVM: SVC(C=0.1, kernel='linear', probability=True, random_state=42)
MLP: MLPClassifier(alpha=0.01, hidden_layer_sizes=(50, 50), random_state=42)

Final Model Performances for combination: ('Bayesian',)

In [ ]:
def run_pipeline(selected_methods, final_data, y_resampled):
    selected_features_names = select_features(selected_methods, final_data , y_resampled)
    X_selected = final_data[selected_features_names] if selected_features_names else final_data.copy()

    X_train, X_test, y_train, y_test = train_test_split(X_selected, y_resampled, test_size=0.2, random_state=42)

    classifiers = {
        "Logistic Regression": LogisticRegression(solver='liblinear', random_state=42),
        "K-Nearest Neighbors": KNeighborsClassifier(),
        "Random Forest": RandomForestClassifier(random_state=42),
        "SVM": SVC(probability=True, random_state=42),
        "MLP": MLPClassifier(random_state=42)
    }

    param_grids = {
        "Logistic Regression": {
            'C': [0.001, 0.01, 0.1, 1, 10, 100],
            'penalty': ['l1', 'l2']
        },
        "K-Nearest Neighbors": {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance']
        },
        "Random Forest": {
            'n_estimators': [100, 200, 300],
            'max_depth': [None, 5, 10, 15]
        },
        "SVM": {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf']
        },
        "MLP": {
            'hidden_layer_sizes': [(50,), (100,), (50, 50)],
            'alpha': [0.0001, 0.001, 0.01]
        }
    }

    for name, clf in classifiers.items():
        param_grid = param_grids.get(name)
        if param_grid:
            grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
            grid_search.fit(X_train, y_train)
            best_params = grid_search.best_params_
            classifiers[name] = clf.set_params(**best_params)
            results = evaluate_model(clf, X_train, X_test, y_train, y_test)
            print(f"{name} after {selected_methods}: Accuracy={results[0]:.2f}, Precision={results[1]:.2f}, Recall={results[2]:.2f}, F1-Score={results[3]:.2f}, AUC={results[4]:.2f}")

    # Use LazyRegressor for regression tasks
    lazy_regressor = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None, random_state=42, regressors='all')
    models, predictions = lazy_regressor.fit(X_train, X_test, y_train, y_test)
    print(models)

    return classifiers


# Preprocess the data

# All combinations of feature selection methods
methods = ['L1', 'L2', 'Bayesian']
all_combinations = []
for r in range(1, len(methods) + 1):
    all_combinations.extend(itertools.combinations(methods, r))

# Perform pipeline for each combination of methods

results = []
for combination in all_combinations:
    print(f"\nRunning pipeline for methods: {combination}")
    classifiers = run_pipeline(combination, final_data , y_resampled)
    results.append((combination, classifiers))

# Further process or display results as needed
for combination, classifiers in results:
    print("\nFinal Model Performances for combination:", combination)
    for name, clf in classifiers.items():
        print(f"{name}: {clf}")


Running pipeline for methods: ('L1',)
Features selected by L1: ['LDA_Component']


AttributeError: 'LogisticRegression' object has no attribute 'parameters'

In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from skopt import gp_minimize
from skopt.space import Integer, Real

# Assuming 'data' is your DataFrame
# Separating the features and target variable
X = data.drop(columns=['is_patient'])
y = data['is_patient']

# Count the number of positive and negative cases
num_positive_cases = (y == 1).sum()
num_negative_cases = (y == 2).sum()

# Calculate the desired number of samples for each class
desired_num_samples = max(num_positive_cases, num_negative_cases)

# Resample the data to balance the classes
ros = RandomOverSampler(sampling_strategy={1: desired_num_samples, 2: desired_num_samples})
X_resampled, y_resampled = ros.fit_resample(X, y)

# Change class label 2 to 0 in y_resampled
y_resampled[y_resampled == 2] = 0

# Count the number of positive and negative cases after resampling
num_positive_cases_resampled = (y_resampled == 1).sum()
num_negative_cases_resampled = (y_resampled == 0).sum()

# Creating a new DataFrame with the resampled data
resampled_data = pd.DataFrame(X_resampled, columns=X.columns)
resampled_data['is_patient'] = y_resampled

# Impute missing values with the median
imputer = SimpleImputer(strategy='median')
data_imputed = pd.DataFrame(imputer.fit_transform(resampled_data), columns=resampled_data.columns)

# Detect and handle outliers
z_scores = np.abs((data_imputed - data_imputed.mean()) / data_imputed.std())
outliers = (z_scores > 3)
for column in data_imputed.columns:
    median = data_imputed[column].median()
    data_imputed.loc[outliers[column], column] = median

data_no_outliers = data_imputed

# Standardize the data
scaler = StandardScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data_no_outliers), columns=data_no_outliers.columns)

# Perform feature extraction using Bayesian optimization and L1/L2 regularization
def objective(params):
    selected_features, C, penalty = params[:-2], params[-2], params[-1]
    selected_features = np.array(selected_features, dtype=bool)
    X_selected = data_scaled.iloc[:, selected_features]

    penalty = 'l1' if penalty < 0.5 else 'l2'
    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    scores = cross_val_score(model, X_selected, y_resampled, cv=5, scoring='accuracy')
    return -np.mean(scores)

# Define the search space for Bayesian optimization
n_features = data_scaled.shape[1]
search_space = [Integer(0, 1) for _ in range(n_features)] + [Real(1e-6, 1e+6, prior='log-uniform'), Real(0, 1)]

# Perform Bayesian optimization
result = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=50,
    random_state=42
)

# Get the best features and parameters
best_features = np.array(result.x[:-2], dtype=bool)
best_C = result.x[-2]
best_penalty = 'l1' if result.x[-1] < 0.5 else 'l2'
selected_features_indices = np.arange(n_features)[best_features]
selected_features_names = [f'feature_{i}' for i in selected_features_indices]

print("Selected Features after Bayesian Optimization and Regularization:")
for feature_name in selected_features_names:
    print(feature_name)
print(f"Best C: {best_C}")
print(f"Best Penalty: {best_penalty}")

# Select the best features
X_selected = data_scaled.iloc[:, selected_features_indices]

# Apply Factor Analysis (FA)
fa = FactorAnalysis(n_components=10)  # Adjust as needed
X_fa = fa.fit_transform(X_selected)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_fa, y_resampled, test_size=0.2, random_state=42)

# Initialize classifiers
log_reg = LogisticRegression(C=best_C, penalty=best_penalty, solver='liblinear', random_state=42)
knn = KNeighborsClassifier()
rf = RandomForestClassifier(random_state=42)
svm = SVC(probability=True, random_state=42)
mlp = MLPClassifier(random_state=42)

# Function to evaluate model
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else np.zeros_like(y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    return accuracy, precision, recall, f1, auc

# Train-test split evaluation
models = {
    "Logistic Regression": log_reg,
    "K-Nearest Neighbors": knn,
    "Random Forest": rf,
    "SVM": svm,
    "MLP": mlp
}

train_test_results = {}
for name, model in models.items():
    results = evaluate_model(model, X_train, X_test, y_train, y_test)
    train_test_results[name] = results

# 10-fold cross-validation evaluation
def cross_val_evaluate_model(model, X, y):
    accuracy = cross_val_score(model, X, y, cv=10, scoring='accuracy').mean()
    precision = cross_val_score(model, X, y, cv=10, scoring='precision').mean()
    recall = cross_val_score(model, X, y, cv=10, scoring='recall').mean()
    f1 = cross_val_score(model, X, y, cv=10, scoring='f1').mean()
    auc = cross_val_score(model, X, y, cv=10, scoring='roc_auc').mean()
    return accuracy, precision, recall, f1, auc

cross_val_results = {}
for name, model in models.items():
    results = cross_val_evaluate_model(model, X_fa, y_resampled)
    cross_val_results[name] = results

# Ensemble classifier (example with majority voting)
ensemble = VotingClassifier(estimators=[
    ('log_reg', log_reg),
    ('knn', knn),
    ('rf', rf),
    ('svm', svm),
    ('mlp', mlp)
], voting='soft')

# Evaluate ensemble classifier
ensemble_results = evaluate_model(ensemble, X_train, X_test, y_train, y_test)
ensemble_cross_val_results = cross_val_evaluate_model(ensemble, X_fa, y_resampled)

# Print results
print("Train-Test Split Results")
for name, results in train_test_results.items():
    print(f"{name}: Accuracy={results[0]:.2f}, Precision={results[1]:.2f}, Recall={results[2]:.2f}, F1-Score={results[3]:.2f}, AUC={results[4]:.2f}")

print("\n10-Fold Cross-Validation Results")
for name, results in cross_val_results.items():
    print(f"{name}: Accuracy={results[0]:.2f}, Precision={results[1]:.2f}, Recall={results[2]:.2f}, F1-Score={results[3]:.2f}, AUC={results[4]:.2f}")

print("\nEnsemble Classifier Results (Train-Test Split):")
print(f"Accuracy={ensemble_results[0]:.2f}, Precision={ensemble_results[1]:.2f}, Recall={ensemble_results[2]:.2f}, F1-Score={ensemble_results[3]:.2f}, AUC={ensemble_results[4]:.2f}")

print("\nEnsemble Classifier Results (10-Fold Cross-Validation):")
print(f"Accuracy={ensemble_cross_val_results[0]:.2f}, Precision={ensemble_cross_val_results[1]:.2f}, Recall")


Selected Features after Bayesian Optimization and Regularization:
feature_1
feature_2
feature_4
feature_5
feature_6
feature_7
feature_8
feature_9
feature_10
Best C: 382880.8868165905
Best Penalty: l2


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Train-Test Split Results
Logistic Regression: Accuracy=0.83, Precision=0.82, Recall=0.79, F1-Score=0.80, AUC=0.91
K-Nearest Neighbors: Accuracy=0.91, Precision=0.93, Recall=0.87, F1-Score=0.90, AUC=0.97
Random Forest: Accuracy=0.94, Precision=0.92, Recall=0.95, F1-Score=0.93, AUC=0.98
SVM: Accuracy=0.88, Precision=0.89, Recall=0.84, F1-Score=0.86, AUC=0.93
MLP: Accuracy=0.83, Precision=0.79, Recall=0.85, F1-Score=0.82, AUC=0.95

10-Fold Cross-Validation Results
Logistic Regression: Accuracy=0.87, Precision=0.90, Recall=0.84, F1-Score=0.87, AUC=0.93
K-Nearest Neighbors: Accuracy=0.88, Precision=0.91, Recall=0.86, F1-Score=0.88, AUC=0.96
Random Forest: Accuracy=0.96, Precision=0.99, Recall=0.94, F1-Score=0.96, AUC=0.99
SVM: Accuracy=0.89, Precision=0.92, Recall=0.86, F1-Score=0.89, AUC=0.93
MLP: Accuracy=0.89, Precision=0.89, Recall=0.89, F1-Score=0.89, AUC=0.95

Ensemble Classifier Results (Train-Test Split):
Accuracy=0.89, Precision=0.87, Recall=0.88, F1-Score=0.87, AUC=0.97

Ensemble 

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
